<a href="https://colab.research.google.com/github/fidelgabriel21/IA_PC1/blob/main/MusicData_FromWeb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtención de datos de la web
En este cuaderno se mostrará como obtener datos relacionada a la industria de la música del repositorio de Jamendo http://dbtune.org/jamendo/ , utilizando rdflib.

In [ ]:
%pip install rdflib

Se trabajará con la información de todos los artista que nos brinda Jamendo, con esta crearemos el grafo


In [ ]:
from rdflib import Graph

j = Graph()
j.parse("http://dbtune.org/jamendo/all/artist")

print(j.serialize(format="turtle").decode("utf-8"))

Cada subject del grafo j, noda indicará una URIRef con información de un determinado artista, para obtener esta data podemos crear múltiples grafos que representen a cada artista

In [ ]:
i = 0 
a = Graph()
for s,o in j.subject_objects():
    i = i + 1 
    if(i == 10): #solo los 10 primeros
        break
    id = s.split('/')
    a.parse(f'http://dbtune.org/jamendo/artist/{id[5]}')
print(a.serialize(format="turtle").decode("utf-8"))

Ahora en el grafo a, tenemos datos sobre diferentes artistas que se pueden extraer mediante un filtro, un ejemplo de esto 
sería extraer name.

In [ ]:
from rdflib.namespace import FOAF

print('Artists names:',end = '\n\n')
for s,o in a.subject_objects(FOAF.name):
  print(o)

In [ ]:
#o las URI de los records que han hecho
print('All records made by Artists:',end = '\n\n')

for s,o in a.subject_objects(FOAF.made):
  print(o)

In [ ]:
#Analizando un record
r = Graph()
r.parse("http://dbtune.org/jamendo/record/2190")
print(r.serialize(format="turtle").decode("utf-8"))

En este ejemplo mostraremos que tambien podemos guardar la data que se está mostrando en una estructura básica de python, se utilizará un dictionario en el que se mostrará un artist y sus records que tiene

In [ ]:

Data = {}
for s,p,o in a.triples((None,FOAF.name,None)):
  id = s.split('/')[5]
  Data[id] = []
  Data[id].append(o)
for s,p,o in a.triples((None,FOAF.made,None)):
  id = s.split('/')[5]
  Data[id].append(o)

for key,item in Data.items():
  for i in range(len(item)):
    if i == 0:
      print(f'The Artist name {item[i]} made:',end='')
    else:
      print(item[i],end=' ')
  print()




# Modelado del Grafo

In [ ]:
from rdflib import URIRef, BNode, Literal
from rdflib import Namespace
from rdflib.namespace import RDF, RDFS, FOAF, DC
from rdflib import ConjunctiveGraph

Definiendo los namespace que usaremos

In [ ]:
ns1 = URIRef("http://purl.org/ontology/mo/")
ns2 = URIRef("http://www.holygoat.co.uk/owl/redwood/0.1/tags/")
owl = URIRef("http://www.w3.org/2002/07/owl#")
dc  = URIRef("http://purl.org/dc/elements/1.1/")

Cargando el archivo music.rdf

In [ ]:
g = ConjunctiveGraph()
g.parse("music.rdf", format="xml")

In [ ]:
len(g)

In [ ]:
print(g.serialize(format="turtle").decode("utf-8"))

# Inferencias

## Primera inferencia
Encontrar los Tracks que fueron hechos por cada Artist

In [ ]:
for subject, record in g.subject_objects(FOAF.made):
  for track in g.transitive_objects(record, ns1 + "track"):
    #print(str(subject) + " made " + str(track))
    print(g.value(subject, FOAF.name) + " made " + g.value(track, DC.title))

## Segunda inferencia
Encontrar los tipos de música (Tags) a los que pertence la producción de cada Artist

In [ ]:
for subject, record in g.subject_objects(FOAF.made):
  for tag in g.transitive_objects(record, ns2 + "taggedWithTag"):
    #print(str(subject) + " made " + str(tag))
    print(g.value(subject, FOAF.name) + " made " + g.value(tag, ns2 + "tagName") + " music")

## Tercera inferencia
Encontrar las licencias bajo las cuales cada Artist lanza su producción

In [ ]:
for subject, record in g.subject_objects(FOAF.made):
  for track in g.transitive_objects(record, ns1 + "track"):
    #print(str(subject) + " made " + str(track))
    print(g.value(subject, FOAF.name) + " made work under " + g.value(track, ns1 + "license") + " license")

##Cuarta Inferencia
Encontrar los track numbers de cada track realizado por un artista

In [ ]:

for subject, record in g.subject_objects(FOAF.made):
  for track in g.transitive_objects(record, ns1 + "track"):
    print(g.value(subject, FOAF.name) + " made " + g.value(track, DC.title) +" that have " + g.value(track, ns1 + "track_number") + " track number")